In [1]:
import sys,os,glob,copy
import numpy as np
import pyslha
import ROOT
import seaborn as sns
from matplotlib import pyplot as plt
from scipy.interpolate import interp1d,griddata,LinearNDInterpolator

ROOT.gSystem.Load(os.path.abspath("./MG5/Delphes/libDelphes.so"))

try:
    ROOT.gInterpreter.Declare('#include "classes/SortableObject.h"')
    ROOT.gInterpreter.Declare('#include "classes/DelphesClasses.h"')
    ROOT.gInterpreter.Declare('#include "external/ExRootAnalysis/ExRootTreeReader.h"')
except:
    pass

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": ["Helvetica"]})

plt.rcParams.update({"savefig.dpi" : 300}) #Figure resolution


#Define plotting style:
sns.set() #Set style
sns.set_style('ticks',{'font.family':'Times New Roman', 'font.serif':'Times New Roman'})
sns.set_context('paper', font_scale=1.8)
cm = plt.cm.get_cmap('RdYlBu')

Welcome to JupyROOT 6.24/06


In [2]:
inputFile = './recastCode/ATLAS_data/HEPData-ins2080541-v1-csv/MuonReconstructionEfficiencydistribution.csv'
gridPtsReco = np.genfromtxt(inputFile,names=True,skip_header=7,delimiter=',')
muonEff_F = LinearNDInterpolator((gridPtsReco['beta'],gridPtsReco['eta']),gridPtsReco['Efficiency'],fill_value=0.0)

inputFile = './recastCode/ATLAS_data/HEPData-ins2080541-v1-csv/TriggerEfficiencydistribution.csv'
gridPtsTrig = np.genfromtxt(inputFile,names=True,skip_header=7,delimiter=',')
triggerEff_F = interp1d(gridPtsTrig['Truth_Level_ETmiss_calo_GeV'],gridPtsTrig['Efficiency'],
                      fill_value=0.0,bounds_error=False)

inputFile = './recastCode/ATLAS_data/HEPData-ins2080541-v1-csv/EventSelectionEfficiencydistribution.csv'
gridPtsSel = np.genfromtxt(inputFile,names=True,skip_header=7,delimiter=',')
selectionEff_F = interp1d(gridPtsSel['Truth_Level_ETmiss_Total_GeV'],gridPtsSel['Efficiency'],
                      fill_value=0.0,bounds_error=False)

inputFile = './recastCode/ATLAS_data/HEPData-ins2080541-v1-csv/TrackSelectionEfficiencydistribution.csv'
gridPtsHigh = np.genfromtxt(inputFile,names=True,skip_header=34,delimiter=',')
trackEffHigh_F = interp1d(gridPtsHigh['beta__gamma'],gridPtsHigh['Efficiency'],
                      fill_value=0.0,bounds_error=False)
gridPtsLow = np.genfromtxt(inputFile,names=True,skip_header=8,skip_footer=32-8,delimiter=',')
trackEffLow_F = interp1d(gridPtsLow['beta__gamma'],gridPtsLow['Efficiency'],
                      fill_value=0.0,bounds_error=False)



def getMuonRecoEff(beta,eta,pid=None):
    """
    Return the interpolated muon reconstruction efficiency
    as a function of beta and eta. If the pid is 
    
    :param beta: HSCP beta
    :param eta: HSCP eta
    
    :return: Efficiency
    """
    
    eta = abs(eta)
    
    eff = muonEff_F(beta,eta)
    
    return float(eff)
    

def getTriggerEff(metCalo):
    
    if metCalo > 1000.:
        return 1.0
    eff = triggerEff_F(metCalo)
    return float(eff)


def getSelectionEff(met):
    
    if met > 1000.:
        return 1.0
    eff = selectionEff_F(met)
    return float(eff)

def getTrackEff(gbeta,sr):
    
    if sr.lower() == 'high':
        eff = trackEffHigh_F(gbeta)
    elif sr.lower() == 'low':
        eff = trackEffLow_F(gbeta)
    else:
        return None
    
    return float(eff)
    

In [3]:
# gbetaVals = np.linspace(0.,3.,1000)
# eff = [getTrackEff(x,sr='low') for x in gbetaVals]

In [4]:
# plt.plot(gbetaVals,eff)
# plt.scatter(gridPtsLow['beta__gamma'],gridPtsLow['Efficiency'])
# plt.yscale('log')
# # plt.ylim(0,1e-3)
# # plt.xlim(0,100)
# plt.show()

In [5]:
parameters = pyslha.readSLHAFile('./pp2C1C1/Cards/param_card.dat')
mLLP = parameters.blocks['MASS'][1000024]
mDM = parameters.blocks['MASS'][1000022]
width = parameters.decays[1000024].totalwidth
if width:
    tau_ns = (6.582e-25/width)*1e9
else:
    tau_ns = np.inf

In [6]:
inputFiles = {'stable' : './pp2C1C1/Events/run_02/chargino_delphes_events.root',
              'prompt' : './pp2C1C1/Events/run_03/prompt_delphes_events.root'}
xsecsPB = {'stable' : 7.133e-3, 'prompt' : 7.133e-3}

### Loop over events, apply basic selection criterium and compute masses

In [7]:
weightsDict = {}
massesDict = {}
sr

for label,inputFile in inputFiles.items():

    f = ROOT.TFile(inputFile,'read')
    tree = f.Get("Delphes")
    nevts = tree.GetEntries()
    
    totalWeight = 0.0
    totalWeight60 = 0.0
    srWeights = {'High': 0.0, 'Low' : 0.0}

    weights = []
    masses = []
    for ievt in range(nevts):
        tree.GetEntry(ievt)
        weight = tree.Weight.At(0).Weight
        totalWeight += weight
        met = tree.MissingET.At(0).MET
        metCalo = tree.MissingETCalo.At(0).MET
        hscps = []
        for iptc in range(tree.Particle.GetEntries()):
            particle = tree.Particle.At(iptc)
            # Get HSCP
            if abs(particle.Charge) != 1: # Skip neutral particles
                continue
            if particle.Mass < 20.: # Skip light (SM) particles
                continue
            if particle.Status != 1: # If HSCP is unstable check if it is the last step
                d1 = particle.D1
                d2 = particle.D2
                daughters = [tree.Particle.At(d) for d in range(d1,d2+1)]
                if any(daughter.PID == particle.PID for daughter in daughters):
                    continue
                # Get the HSCP decay radius from the first daughter production vertex:
                r_decay = np.sqrt(daugthers[0].X**2 +daugthers[0].Y**2)
                particle.r_decay = r_decay
            else:
                particle.r_decay = np.inf
            hscps.append(particle)
            
        # Get HSCPs pT
        px = sum([hscp.Px for hscp in hscps])
        py = sum([hscp.Py for hscp in hscps])
        hscpPT = np.sqrt(px**2 + py**2)
        if hscpPT > 60.:
            totalWeight60 += weight
        
        muonsLLP = []
        # Get probability of tagging the HSCP as muon:
        for hscp in hscps:
            if hscp.r_decay < 1e4: # Skip decays before MS
                continue
            beta = np.sqrt(hscp.Px**2 +hscp.Py**2 + hscp.Pz**2)/hscp.E
            eta = abs(hscp.Eta)
            eff = getMuonRecoEff(beta,eta,hscp.PID)
            # Randomly reconstrunct the HSCP as a muon
            if np.random.uniform() > eff:
                continue
            muonsLLP.append(hscp)
        
        if muonsLLP:
            # Remove LLPs reconstructed as muons from the hscp list:
            hscps = [hscp for hscp in hscps if hscp not in muonsLLP]
            # Remove muonsLLp from MET:            
            pxTot = sum([m.Px for m in muonsLLP])
            pyTot = sum([m.Py for m in muonsLLP])
            metx = met*np.cos(tree.MissingET.At(0).Phi)
            mety = met*np.sin(tree.MissingET.At(0).Phi)
            newMet = np.sqrt((metx-pxTot)**2 + (mety-pyTot)**2)
            met = newMet
        
        if not hscps:
            continue
        triggerEff = getTriggerEff(metCalo)
        eventEff = getSelectionEff(met)
        
        hscpEffs = {'SR-Inclusive_High' : [], 'SR-Inclusive_Low' : []}
        for hscp in hscps:
            if hscp.PT < 120.:
                continue
            if abs(hscp.Eta) > 1.8:
                continue
            if hscp.r_decay < 500.0:
                continue
            gbeta = hscp.E/hscp.Mass
            trackEffHigh = getTrackEff(gbeta,sr='High')
            trackEffLow = getTrackEff(gbeta,sr='Low')
            wMassLow = 0.6
            wMassHigh = max(0.,0.74 - 0.052*(hscp.Mass/1000.))
            hscpEffs['SR-Inclusive_High'].append(trackEffHigh*wMassHigh)
            hscpEffs['SR-Inclusive_Low'].append(trackEffLow*wMassLow)
            
        
        # Total event weight
        hscpLow = np.array(hscpEffs['SR-Inclusive_Low'])
        eventWeightLow = weight*triggerEff*eventEff*(1-np.prod(1.0-hscpLow))

        hscpHigh = np.array(hscpEffs['SR-Inclusive_High'])
        eventWeightHigh = weight*triggerEff*eventEff*(1-np.prod(1.0-hscpHigh))
        
        srWeights['Low'] += eventWeightLow
        srWeights['High'] += eventWeightHigh

    # weights = np.array(weights)
    # masses = np.array(masses)
    # weightsDict[label] = weights
    # massesDict[label] = masses
print('Total weight = ',totalWeight)
print('SR=',srWeights)

Total weight =  355.90219777077436
SR= {'High': 0.28915631553555915, 'Low': 2.0855278872503407}


In [8]:
for sr,eff in srWeights.items():
    print('%s eff = %1.4f' %(sr,eff/totalWeight))
    print('%s eff = %1.4f' %(sr,eff/totalWeight60))
#

High eff = 0.0008
High eff = 0.0019
Low eff = 0.0059
Low eff = 0.0138


### Remove failed attempts to reconstruct masses

In [ ]:
for label in massesDict:
    oldMasses = massesDict[label]
    oldWeights = weightsDict[label]

    newMasses = []
    newWeights = []
    for i,mass in enumerate(oldMasses):
        # Remove events where reconstruction failed
        if np.any(mass == None): 
            continue
        # Remove events for which parent masses differ more than 5%
        if 2*abs(mass[0,0]-mass[1,0])/(mass[0,0]+mass[1,0]) > 0.05: 
            continue
        # Remove events for which parent masses are smaller than daughers        
        if (mass[0,0] < mass[0,1]) or (mass[1,0] < mass[1,1]): 
            continue
        newMasses.append(mass)
        newWeights.append(oldWeights[i])
        
    massesDict[label] = np.array(newMasses)
    weightsDict[label] = np.array(newWeights)

### Check how many events were kept:

In [ ]:
for label in massesDict:
    f = ROOT.TFile(inputFiles[label],'read')
    tree = f.Get("Delphes")
    nevts = tree.GetEntries()
    kept = len(massesDict[label])
    print('\n%s: Kept %i events out of %i' %(label,kept,nevts))

### Plots

In [ ]:
plt.figure(figsize=(9,5))

colorsDict = dict(zip(massesDict.keys(),sns.color_palette("Paired")[:len(massesDict)]))
lineDict = dict(zip(massesDict.keys(),['-','--']))

for label in massesDict:
    m = massesDict[label]
    
    motherMasses = np.concatenate((m[:,0,0],m[:,1,0]))
    daughterMasses = np.concatenate((m[:,0,1],m[:,1,1]))


    plt.hist(motherMasses,bins=np.arange(0,600.,10.),
             histtype='step',label=r'Mother (%s)' %label,
             linewidth=2,color=colorsDict[label],linestyle=lineDict[label])
    
    plt.hist(daughterMasses,bins=np.arange(0,600.,10.),
            histtype='step',label=r'Daughter (%s)' %label,
            linewidth=2,color=colorsDict[label],linestyle=lineDict[label])


# plt.yscale('log')
plt.xlabel(r'$M_X$ (GeV)')
plt.ylabel('Events')
plt.legend(loc=(0.9,0.6),framealpha=1.0)
plt.title(r'$M_{F} = %1.0f$ GeV, $M_{s} = %1.0f$ GeV, $\lambda_{3} = %1.1f$'
          %(mF,mDM,ls3))
plt.tight_layout()
plt.show()

In [ ]:
lumi = 3000e3 # luminosity in 1/pb
lsEff = 4*mDM

allProcessesM = []
allProcessesD = []
allProcessesW = []
allProcessesL = []
for label in massesDict:
    m = massesDict[label]
    w = weightsDict[label]
    
    motherMasses = np.concatenate((m[:,0,0],m[:,1,0]))
    daughterMasses = np.concatenate((m[:,0,1],m[:,1,1]))
    weights = ((lsEff/ls3)**2)*np.concatenate((w,w))*lumi # Rescale weights by new lambda3 value

    allProcessesM.append(motherMasses)
    allProcessesD.append(daughterMasses)
    allProcessesW.append(weights)
    allProcessesL.append(label)
    
plt.figure(figsize=(10,5))
label = 'Mother (' +','.join(allProcessesL) +')'
color = [colorsDict[l] for l in allProcessesL]
bins = plt.hist(allProcessesM,bins=np.arange(0,600.,10.),histtype='bar',stacked=True,
                label=label,linewidth=2,
                weights=allProcessesW,color=color)

label = 'Daughter (' +','.join(allProcessesL) +')'
bins = plt.hist(allProcessesD,bins=np.arange(0,600.,10.),histtype='bar',stacked=True,
                label=label,linewidth=2,
                weights=allProcessesW,color=color)


ymax = bins[0].flatten().max()    
    
plt.yscale('log')
plt.ylim(1,2*ymax)
plt.xlabel(r'$M_X$ (GeV)')
plt.ylabel('Events')
plt.legend(loc=(0.9,0.6),framealpha=1.0)
plt.title(r'$M_{F} = %1.0f$ GeV, $M_{s} = %1.0f$ GeV, $\lambda_{3} = %1.1f$ GeV, $\mathcal{L} = %1.0f \mbox{ fb}^{-1}$' %(mF,mDM,lsEff,lumi/1e3))
plt.tight_layout()
plt.show()